In [ ]:
import scipy
from scipy.integrate import simps
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import os,sys 
os.chdir("..")

In [ ]:
from torchquad.utils.benchmark_utils import _runtime_measure as runtime_measure
from torchquad.utils.benchmark_utils import _get_integral as get_integral
from torchquad.plots import plot_runtime as plt_rt
from torchquad.plots import plot_convergence as plt_conv
from torchquad.integration.monte_carlo import MonteCarlo 
from torchquad.integration.trapezoid import Trapezoid
from torchquad.integration.simpson import Simpson
from torchquad.utils.enable_cuda import enable_cuda
from torchquad.tests.integration_test_functions import Polynomial, Exponential, Sinusoid
import torch
torch.set_default_tensor_type(torch.DoubleTensor)

## Enable GPU usage

In [ ]:
enable_cuda()

## Set precision

In [ ]:
np.set_printoptions(precision=10)
torch.set_printoptions(precision=10)

## Some test functions

In [ ]:
def f1(x):
    return Polynomial(coeffs=[4,0,-1], dim=1)._poly(x) #4-x^2

def f2(x):
    return Polynomial(coeffs=[1,-3,4,23,-4,1], dim=1)._poly(x)#x^5-4*x^4+23*x^3+4*x^2-3*x+1

def f1_3D(x):
    return Polynomial(coeffs=[4,0,-1], dim=3)._poly(x)

def f2_3D(x):
    return Polynomial(coeffs=[1,-3,4,23,-4,1], dim=3)._poly(x)


## Selecting test function and integration method

In [ ]:
f_test_dict={'f1' : f1, 'f2' : f2}
f_test_3D_dict={'f1' : f1_3D, 'f2' : f2_3D}

In [ ]:
f_test_used = 'f1'
f_test_1D = f_test_dict[f_test_used]
f_test_3D = f_test_3D_dict[f_test_used]

In [ ]:
trapz_scipy_method = scipy.trapz #scipy_based trapezoid integration method
trapz_torch_method = Trapezoid() #Torchquad trapezoid integration method
simpson_scipy_method = simps #scipy_based simpson integration method
simpson_torch_method = Simpson() #Torchquad simpson integration method

## Selecting number of integration points and average measurements per step

In [ ]:
N_points = [11, 101, 1001, 10001] #Number of iteration tests
iterations = 10 #Number of iterations used to take an average runtime measurement

In [ ]:
ground_through_dict = {'f1' : 22/3 * np.ones(len(N_points)), 'f1_2' : 46/15 * np.ones(len(N_points)), 'f_x': np.zeros(len(N_points))}

## Testing runtime performance

In [ ]:
scipy_trapz1D_runtime = runtime_measure(method=trapz_scipy_method, dim=1, scipy_based=True, fn=f_test_1D, N=N_points, iterations=iterations)

In [ ]:
torch_trapz1D_runtime = runtime_measure(method=trapz_torch_method.integrate, dim=1, scipy_based=False, fn=f_test_1D, N=N_points, iterations=iterations)

In [ ]:
scipy_simps_runtime = runtime_measure(method=simpson_scipy_method, dim=1, scipy_based=True, fn=f_test_1D, N=N_points, iterations=iterations)

In [ ]:
torch_simps_runtime = runtime_measure(method=simpson_torch_method.integrate, dim=1, scipy_based=False, fn=f_test_1D, N=N_points, iterations=iterations)

In [ ]:
scipy_3d_quad = runtime_measure(method=simpson_torch_method.integrate, dim=3, scipy_based=True, fn=f_test_3D, N=N_points, iterations=iterations)

In [ ]:
plt_rt.plot_runtime([N_points, N_points, N_points, N_points], [scipy_trapz1D_runtime, torch_trapz1D_runtime, scipy_simps_runtime, torch_simps_runtime], labels=['scipy_trapz1D', 'torch_trapz1D', 'scipy_simps', 'torch_simps'])

## Measuring convergence rate

In [ ]:
scipy_method_integral = get_integral(method=trapz_scipy_method, dim=1, scipy_based=True, fn=f_test, N=N_points)

In [ ]:
torch_method_integral = get_integral(method=integral_torch_method.integrate, scipy_based=False, fn=f_test, N=N_points)

In [ ]:
plt_conv.plot_convergence([N_points, N_points], [scipy_method_integral, torch_method_integral], ground_through_dict[f_test_used] , ['scipy_method', 'torch_method'], dpi=150)